# Applied Data Science Capstone


### Problem 1.1 Introduction/Business Problem



#### As a student living in Williamsburg, VA, I feel that restaurants are not enough, especially Chinese restaurants. People in Williamsburg deserve to try Chinese famous type of food - Hot Pot. Therefore, this capstone is going to find the best location to open a restaurants according to the location's reviews, popularity and other factors. The data will be scraped from Foursquare and the problem will be solved using Foursquare API.

### Problem 1.2 Data 

#### The data is from Foursquare for Williamsburg, looking for the best location in Williamsburg

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\123\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         860

In [29]:
CLIENT_ID = 'IIM3ZPX4O0MTECO31PIVRFK0UQMYYAJNVRWABOAJLCYEFRF0' # your Foursquare ID
CLIENT_SECRET = 'KVV1LN5ITDZ4YDV3FGXM2WLSOL5OVSPMR3GQPYFNIOFACQBO' # your Foursquare Secret
VERSION = '20200402'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IIM3ZPX4O0MTECO31PIVRFK0UQMYYAJNVRWABOAJLCYEFRF0
CLIENT_SECRET:KVV1LN5ITDZ4YDV3FGXM2WLSOL5OVSPMR3GQPYFNIOFACQBO


In [30]:
address = 'Williamsburg, VA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.27892105 -76.69448606443117


In [31]:
search_query = 'Restaurant'
radius = 5000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [32]:
# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IIM3ZPX4O0MTECO31PIVRFK0UQMYYAJNVRWABOAJLCYEFRF0&client_secret=KVV1LN5ITDZ4YDV3FGXM2WLSOL5OVSPMR3GQPYFNIOFACQBO&ll=37.27892105,-76.69448606443117&v=20200402&query=Restaurant&radius=5000&limit=100'

In [33]:
# Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e86917947b43d002363eb8a'},
 'response': {'venues': [{'id': '4b5b270cf964a520e3e628e3',
    'name': 'The Trellis Restaurant',
    'location': {'address': '403 W Duke of Gloucester St',
     'crossStreet': 'N Henry St',
     'lat': 37.27108130431542,
     'lng': -76.70615589922397,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.27108130431542,
       'lng': -76.70615589922397}],
     'distance': 1352,
     'postalCode': '23185',
     'cc': 'US',
     'city': 'Williamsburg',
     'state': 'VA',
     'country': 'United States',
     'formattedAddress': ['403 W Duke of Gloucester St (N Henry St)',
      'Williamsburg, VA 23185',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': 

In [35]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4b5b270cf964a520e3e628e3,The Trellis Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1585877339,False,403 W Duke of Gloucester St,N Henry St,37.271081,-76.706156,"[{'label': 'display', 'lat': 37.27108130431542...",1352,23185,US,Williamsburg,VA,United States,"[403 W Duke of Gloucester St (N Henry St), Wil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4bb0e18ff964a5203c663ce3,Berret's Seafood Restaurant and Taphouse Grill,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1585877339,False,199 S Boundary St,Merchant's Square,37.270027,-76.707125,"[{'label': 'display', 'lat': 37.27002662899438...",1494,23185,US,Williamsburg,VA,United States,"[199 S Boundary St (Merchant's Square), Willia...",2044294,https://www.grubhub.com/restaurant/berrets-sea...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,36256198
2,4c73ec6aec8d952110def1c2,Peking Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1585877339,False,120 Waller Mill Rd Ste J,Bypass Rd,37.288596,-76.717956,"[{'label': 'display', 'lat': 37.28859584124185...",2341,23185,US,Williamsburg,VA,United States,"[120 Waller Mill Rd Ste J (Bypass Rd), William...",NaN,NaN,NaN,NaN,NaN,NaN,445345761
3,4daa52174df01c19b1886570,The Veranda Restaurant (Fort Magruder Hotel),"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1585877339,False,NaN,NaN,37.263629,-76.679951,"[{'label': 'display', 'lat': 37.26362902781627...",2134,NaN,US,Williamsburg,VA,United States,"[Williamsburg, VA, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4c3243067cc0c9b63005f09a,The Jefferson Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1585877339,False,1453 Richmond Rd,NaN,37.289970,-76.722190,"[{'label': 'display', 'lat': 37.28997, 'lng': ...",2744,23185,US,Williamsburg,VA,United States,"[1453 Richmond Rd, Williamsburg, VA 23185, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,The Trellis Restaurant,American Restaurant,403 W Duke of Gloucester St,N Henry St,37.271081,-76.706156,"[{'label': 'display', 'lat': 37.27108130431542...",1352,23185,US,Williamsburg,VA,United States,"[403 W Duke of Gloucester St (N Henry St), Wil...",4b5b270cf964a520e3e628e3
1,Berret's Seafood Restaurant and Taphouse Grill,Seafood Restaurant,199 S Boundary St,Merchant's Square,37.270027,-76.707125,"[{'label': 'display', 'lat': 37.27002662899438...",1494,23185,US,Williamsburg,VA,United States,"[199 S Boundary St (Merchant's Square), Willia...",4bb0e18ff964a5203c663ce3
2,Peking Restaurant,Japanese Restaurant,120 Waller Mill Rd Ste J,Bypass Rd,37.288596,-76.717956,"[{'label': 'display', 'lat': 37.28859584124185...",2341,23185,US,Williamsburg,VA,United States,"[120 Waller Mill Rd Ste J (Bypass Rd), William...",4c73ec6aec8d952110def1c2
3,The Veranda Restaurant (Fort Magruder Hotel),American Restaurant,NaN,NaN,37.263629,-76.679951,"[{'label': 'display', 'lat': 37.26362902781627...",2134,NaN,US,Williamsburg,VA,United States,"[Williamsburg, VA, United States]",4daa52174df01c19b1886570
4,The Jefferson Restaurant,American Restaurant,1453 Richmond Rd,NaN,37.289970,-76.722190,"[{'label': 'display', 'lat': 37.28997, 'lng': ...",2744,23185,US,Williamsburg,VA,United States,"[1453 Richmond Rd, Williamsburg, VA 23185, Uni...",4c3243067cc0c9b63005f09a
5,Captain George's Seafood Restaurant,Seafood Restaurant,5363 Richmond Rd,NaN,37.310893,-76.732369,"[{'label': 'display', 'lat': 37.3108926, 'lng'...",4890,23188,US,Williamsburg,VA,United States,"[5363 Richmond Rd, Williamsburg, VA 23188, Uni...",4b63907cf964a520b6832ae3
6,South of the Border Mexican Restaurant,Mexican Restaurant,322 Second St,NaN,37.273983,-76.683641,"[{'label': 'display', 'lat': 37.27398335639371...",1106,23185,US,Williamsburg,VA,United States,"[322 Second St, Williamsburg, VA 23185, United...",4c28c7f097d00f477c5a40ea
7,Antonio's Italian Restaurant,Italian Restaurant,NaN,NaN,37.295480,-76.686481,"[{'label': 'display', 'lat': 37.29547966834879...",1974,23185,US,Williamsburg,VA,United States,"[Williamsburg, VA 23185, United States]",509ac54de4b05481aa3aa06f
8,"Second Street, An American Bistro",American Restaurant,140 Second St,Page St,37.274577,-76.687106,"[{'label': 'display', 'lat': 37.27457674108086...",813,23185,US,Williamsburg,VA,United States,"[140 Second St (Page St), Williamsburg, VA 231...",4b22a52ef964a520f94a24e3
9,Sal's By Victor,Italian Restaurant,1242 Richmond Rd,NaN,37.280697,-76.720294,"[{'label': 'display', 'lat': 37.28069749297099...",2294,23185,US,Williamsburg,VA,United States,"[1242 Richmond Rd, Williamsburg, VA 23185, Uni...",4c23714e9085d13a6bdc87cc
